# Inference

This notebook shows **how to run inference with the [HyPINO multi-physics neural operator](https://arxiv.org/abs/2509.05117)**: what inputs it expects, what it returns, and how to use the result in downstream tasks (evaluation, visualization, or refinement).

**What HyPINO produces:** Given a PDE specification (PDE coefficients, boundary/initial data, forcing), HyPINO returns a **neural network that represents the solution field**, which enables **continuous evaluation** anywhere in the domain and **analytical derivatives** to compute residuals (via automatic differentiation).

In [ ]:
import sys, os

project_root = os.path.abspath("..")
sys.path.append(project_root)

In [ ]:
import os
import torch
import numpy as np

from src.models import HyPINO
from src.data.utils import to_tensor
from src.data.utils import plot_grids, encode_pde_str
from src.data.utils import ALL_DERIVATIVES, compile_diff_operator, compute_derivatives, plot_grids

In [ ]:
if torch.cuda.is_available():
    device = f'cuda:{torch.cuda.current_device()}'
else:
    device = 'cpu'

## Load Example PDEs

HyPINO provides **7 benchmark PDEs** that can be loaded to explore which inputs are required for inference:

`heat`, `helmholtz`, `helmholtz_G`, `poisson_C`, `poisson_G`, `poisson_L`, `wave`


### Required Inputs

HyPINO expects the following **grid-based inputs**:

- **Dirichlet boundary mask**  
  Binary grid where cells with value `1` indicate Dirichlet boundaries, and `0` otherwise.  
  *Tip:* Best performance is achieved when boundary lines are 2–3 pixels wide.

- **Neumann boundary mask**  
  Binary grid similar to the Dirichlet mask, but indicating Neumann boundaries.

- **Dirichlet boundary values**  
  Grid of actual dirichlet boundary condition values.  
  Will be multiplied by the dirichlet mask, i.e. values outside of boundaries can be set to an arbitrary value (e.g. `0`).

- **Neumann boundary values**  
  Grid of actual neumann boundary condition values.  
  Will be multiplied by the neumann mask, i.e. values outside of boundaries can be set to an arbitrary value (e.g. `0`).

- **Source (forcing) function**  
  The source or forcing term of the PDE, discretized over the grid.


In [ ]:
inputs_path = '../assets/helmholtz/arrays' # change path to load different benchmark

# inputs
dirichlet_mask = np.load(os.path.join(inputs_path, 'dirichlet_mask.npy'))
dirichlet_conditions = np.load(os.path.join(inputs_path, 'dirichlet_conditions.npy'))
neumann_mask = np.load(os.path.join(inputs_path, 'neumann_mask.npy'))
neumann_conditions = np.load(os.path.join(inputs_path, 'neumann_conditions.npy'))
source_function = np.load(os.path.join(inputs_path, 'source_function.npy'))

# if available, load reference solution
reference_solution = np.load(os.path.join(inputs_path, 'reference_solution.npy'))

plot_grids([dirichlet_mask, dirichlet_conditions, neumann_mask, neumann_conditions, source_function, reference_solution], 
           titles=['Dirichlet mask', 'Dirichlet boundary cond', 'Neumann mask', 
                   'Neumann boundary cond', 'Source function', 'Reference solution'])

All grid-based inputs are stacked into a 5-d tensor in this order: dirichlet mask, neumann mask, dirichlet values, neumann values, source function.

In [ ]:
mat_inputs = to_tensor(np.stack([dirichlet_mask, neumann_mask,
                                 dirichlet_conditions * dirichlet_mask, neumann_conditions * neumann_mask,
                                 source_function], axis=0)).to(device)

The PDE coefficients are encoded as vector of coefficients. We provide two methods to convert a string of the operator into a dictionary of coefficients, and then into a tensor.

In [ ]:
diff_operator = 'uxx + uyy + u'
pde_coeffs = encode_pde_str(diff_operator)
print('pde coefficients:       ', pde_coeffs)

pde_coeffs_tensor = to_tensor([c for c in pde_coeffs.values()]).to(device)
print('pde coefficients tensor:', pde_coeffs_tensor)

## Load model

In [ ]:
model = HyPINO.load_from_safetensors('../models/hypino.safetensors').to(device).eval()

## Generate target network

In [ ]:
# call model with pde coeffs and mat_inputs. needs unsqueeze to add the batch dimension, then remove batch dimension with [0]
target_pinn = model(pde_coeffs_tensor.unsqueeze(0), mat_inputs.unsqueeze(0))[0]

The returned object is a ready-to-use function (a `functools.partial`) for making predictions. It already contains the trained weights and biases, so you only need to provide an `(N, 2)` tensor of `(x, y)` coordinates, where `N` is the number of collocation points.

Alternatively, for full control over the model, you can access the weights and biases directly. This is ideal if you plan to write your own optimization routine or need to inspect the model's parameters:

In [ ]:
_, weights, biases = model(pde_coeffs_tensor.unsqueeze(0), mat_inputs.unsqueeze(0), return_weights=True)

And then import them into a PyTorch Module, e.g. our `FNN` class:

In [ ]:
from src.models.utils import FNN
target_pinn = FNN.from_parameters(weights, biases, remove_batch_dim=True)

Now you can do operations like saving and loading the state-dict:

In [ ]:
torch.save(target_pinn.state_dict(), 'generated_pinn.pth')

## Query generated target network

In [ ]:
# generate a grid of 2D collocation points on the domain [-1, 1]^2
x_grid, y_grid = np.meshgrid(
    np.linspace(-1,  1, 224),
    np.linspace( 1, -1, 224),
)
x = to_tensor(x_grid, requires_grad=True).reshape(-1, 1).to(device)
y = to_tensor(y_grid, requires_grad=True).reshape(-1, 1).to(device)
xy = torch.cat([x, y], dim=-1)

In [ ]:
# predict PDE solutions at collocation points
u_pred = target_pinn(xy.unsqueeze(0))[0]
u_pred_grid = u_pred.detach().cpu().numpy().reshape(224, 224)

In [ ]:
# plot against reference solution
plot_grids([
    u_pred_grid,
    reference_solution,
    u_pred_grid - reference_solution,
    ], titles=['Predicted solution', 'Reference solution', 'Difference'])

To evaluate the residual, we need to compute the derivatives of the predicted values in `u_pred` w.r.t. the inputs `x` and `y`. This can be done with the pytorch autograd function. We provide the `compute_derivatives` method for convenience:

In [ ]:
pred_derivs = compute_derivatives(
    in_var_map={'x': x, 'y': y},
    out_var_map={'u': u_pred},
    derivatives=ALL_DERIVATIVES
)
print(pred_derivs.keys())

We provide a convenince method `compile_diff_operator` taking the operator string and converting it into a callable function, with the necessary derivatives as input:

In [ ]:
diff_op = compile_diff_operator(diff_operator)
f_pred = diff_op(pred_derivs)
f_pred_grid = f_pred.detach().cpu().numpy().reshape(224, 224)

In [ ]:
# plot predicted source function and residual
plot_grids([
    f_pred_grid,
    source_function,
    f_pred_grid - source_function,
    ], titles=['Predicted source fn', 'True source fn', 'Difference'])